## Q2: Taking a closer look at the hanging cable (catenary) problem (50 pts)

In [1]:
import Pkg; Pkg.activate(dirname(@__DIR__)); Pkg.instantiate()

  Activating environment at `~/Projects/dynamics_simulation/hw2_solution/hw2_solutions-brucekimrokcmu/Project.toml`
    Updating registry at `~/.julia/registries/General`
   Installed ForwardDiff ────────────────────── v0.10.35
   Installed BlockDiagonals ─────────────────── v0.1.41
   Installed Conda ──────────────────────────── v1.8.0
   Installed ControlSystems ─────────────────── v1.6.1
   Installed Colors ─────────────────────────── v0.12.10
   Installed StaticArrays ───────────────────── v1.5.16
   Installed JLD2 ───────────────────────────── v0.4.31
   Installed OrdinaryDiffEq ─────────────────── v6.33.3
   Installed DifferentialEquations ──────────── v7.6.0
   Installed Plots ──────────────────────────── v1.38.6
   Installed MakieCore ──────────────────────── v0.6.2
   Installed SteadyStateDiffEq ──────────────── v1.12.0
   Installed TriangularSolve ────────────────── v0.1.19
   Installed Rotations ──────────────────────── v1.3.4
   Installed RealDot ────────────────────────── 

   Installed Hungarian ──────────────────────── v0.7.0
   Installed IrrationalConstants ────────────── v0.2.2
    Updating `~/Projects/dynamics_simulation/hw2_solution/hw2_solutions-brucekimrokcmu/Project.toml`
  [0a1fb500] + BlockDiagonals v0.1.41
  [5ae59095] + Colors v0.12.10
  [8f4d0f93] + Conda v1.8.0
  [a6e380b2] + ControlSystems v1.6.1
  [150eb455] + CoordinateTransformations v0.6.2
  [0c46a032] + DifferentialEquations v7.6.0
  [5789e2e9] + FileIO v1.16.0
  [f6369f11] + ForwardDiff v0.10.35
  [5c1252a2] + GeometryBasics v0.3.13
  [b6b21f68] + Ipopt v0.7.0
  [033835bb] + JLD2 v0.4.31
  [b8f27783] + MathOptInterface v0.9.22
  [283c5d60] + MeshCat v0.13.2
  [7269a6da] + MeshIO v0.4.7
  [1dea7af3] + OrdinaryDiffEq v6.33.3
  [f0f68f2c] + PlotlyJS v0.18.10
  [91a5bcdd] + Plots v1.38.6
  [38ceca67] + RobotDynamics v0.3.4
  [74be38bb] + RobotZoo v0.2.3
  [6038ab10] + Rotations v1.3.4
  [90137ffa] + StaticArrays v1.5.16
  [7770976a] + TrajOptPlots v0.2.0
    Updating `~/Projects/dynamics

  [3cdcf5f2] + RecipesBase v1.3.3
  [01d81517] + RecipesPipeline v0.6.11
  [731186ca] + RecursiveArrayTools v2.32.3
  [f2c3362d] + RecursiveFactorization v0.2.18
  [189a3867] + Reexport v1.2.2
  [05181044] + RelocatableFolders v1.0.0
  [ae029012] + Requires v1.3.0
  [ae5879a3] + ResettableStacks v1.1.1
  [708f8203] + Richardson v1.4.0
  [79098fc4] + Rmath v0.7.1
  [38ceca67] + RobotDynamics v0.3.4
  [74be38bb] + RobotZoo v0.2.3
  [6038ab10] + Rotations v1.3.4
  [7e49a35a] + RuntimeGeneratedFunctions v0.5.5
  [94e857df] + SIMDTypes v0.1.0
  [476501e8] + SLEEFPirates v0.6.38
  [0bca4576] + SciMLBase v1.81.0
  [6c6a2e73] + Scratch v1.1.1
  [efcf1570] + Setfield v1.1.1
  [992d4aef] + Showoff v1.0.3
  [727e6d20] + SimpleNonlinearSolve v0.1.5
  [699a6c99] + SimpleTraits v0.9.4
  [66db9d55] + SnoopPrecompile v1.0.3
  [a2af1166] + SortingAlgorithms v1.1.0
  [47a9eef4] + SparseDiffTools v1.30.0
  [e56a9233] + Sparspak v0.3.9
  [276daf66] + SpecialFunctions v2.2.0
  [aedffcd0] + Static v0.8.4
  

    Building Conda → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/e32a90da027ca45d84678b826fffd3110bb3fc90/build.log`
    Building WebIO → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/976d0738247f155d0dcd77607edea644f069e1e9/build.log`


In [2]:
using LinearAlgebra
using ForwardDiff
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Plots
using OrdinaryDiffEq
using FileIO
plotlyjs()

const jld2path = joinpath(dirname(@__DIR__),"testfiles")
include(joinpath(jld2path,"test_functions.jl"))
test_dict = load(joinpath(jld2path,"q2.jld2"));

┌ Info: Precompiling ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
└ @ Base loading.jl:1342
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1342
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1342
┌ Info: Precompiling OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed]
└ @ Base loading.jl:1342
┌ Warning: Replacing docs for `SciMLBase.sol :: Union{Tuple, Tuple{D}, Tuple{S}, Tuple{N}, Tuple{T}} where {T, N, S, D}` in module `SciMLBase`
└ @ Base.Docs docs/Docs.jl:240
┌ Info: Precompiling PlotlyJS [f0f68f2c-4968-5e81-91da-67840de0976a]
└ @ Base loading.jl:1342


WebIO._IJuliaInit()

┌ Info: Precompiling JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1342


## 2A: Solve catenary problem with nonlinear optimization (15 pts)

In [3]:
#Problem parameters (feel free to play with these)
g = 9.81
w = 10 #width of gap
ℓ = 15 #length of cable
m = 10 #mass of cable
n = 25 #number of discrete segments

Δx = w/n #width of each discrete segment
ρ = m/ℓ; #mass density

In [4]:
#Total cable length
function S(y,ẏ)
    s = 0.0
    
    ### Solution
    for k = 1:(n-1)
        s += sqrt(1 + ẏ[k]^2)*Δx
    end
    return s
end

function S(y)
    ẏ = (y[2:end]-y[1:(end-1)])./Δx
    S(y,ẏ)
end

S (generic function with 2 methods)

In [5]:
@testset "cable length" begin 
    y = [1:5;5:-1:1;1:5;10:-1:1]
    
    s1 = S(y)
    
    @test isapprox(s1,test_dict["s1"],atol = 1e-10)
end
    
    

Test Summary: | Pass  Total
cable length  |    1      1


Test.DefaultTestSet("cable length", Any[], 1, false, false)

In [6]:
#Potential energy
function U(y,ẏ)
    u = 0.0
    
    ### Solution
    for k = 1:(n-1)
        ym = 0.5*(y[k+1]+y[k])
        u += ρ*g*ym*sqrt(1 + ẏ[k]^2)*Δx
    end
    return u
end

function U(y)
    ẏ = (y[2:end]-y[1:(end-1)])./Δx
    U(y,ẏ)
end

U (generic function with 2 methods)

In [7]:
@testset "potential energy" begin 
    y = [1:5;5:-1:1;1:5;10:-1:1]
    
    u1 = U(y)
    
    @test isapprox(u1,test_dict["u1"],atol = 1e-10)
end

Test Summary:    | Pass  Total
potential energy |    1      1


Test.DefaultTestSet("potential energy", Any[], 1, false, false)

In [8]:
#End points (feel free to play with these)
y1 = 0.0
yn = 0.0

0.0

In [9]:
#Objective and constraint functions for IPOPT

function objective(z)
    y = z[1:n]
    ẏ = z[(n+1):end]
    
    return U(y,ẏ) #minimize potential energy
end

function constraint!(c,z)
    # this function edits the argument c in place 
    # the ! at the end of a function name in julia usually means this
    
    y = z[1:n]
    ẏ = z[(n+1):end]
    
    c .= [
        NaN #initial point
        NaN #final point
        NaN #length constraint
        NaN*zeros(length(ẏ)) #velocity constraints
    ]
    
    ### Solution
    c .= [
        y[1] - y1 #initial point
        y[end] - yn #final point
        S(y,ẏ) - ℓ #length constraint
        y[2:end] .- y[1:(end-1)] .- ẏ.*Δx #velocity constraints
    ]
    
    return nothing
    
end

constraint! (generic function with 1 method)

In [10]:
@testset "constraint and objective" begin 
    y = [1:5;5:-1:1;1:5;10:-1:1] .- 1
    ẏ = diff(y)/Δx
    
    z = [y;ẏ]
    
    c = zeros(3 + n - 1)
    constraint!(c,z)
    
    @test norm(c[1:2]) < 1e-10 
    @test isapprox(abs(c[3]),13.433666671558711,atol = 1e-5)
    @test norm(c[4:end]) < 1e-10 
end

Test Summary:            | Pass  Total
constraint and objective |    3      3


Test.DefaultTestSet("constraint and objective", Any[], 3, false, false)

In [11]:
#Boilerplate setup code to interface with IPOPT (don't touch this)

struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function primal_bounds(n)
    x_l = -Inf*ones(n)
    x_u = Inf*ones(n)
    return x_l, x_u
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    c_l[idx_ineq] .= -Inf

    c_u = zeros(m)
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    objective(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,objective,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    constraint!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), constraint!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function ipopt_solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-9,c_tol=1.0e-9,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

ipopt_solve (generic function with 1 method)

In [12]:
#Solve with IPOPT
n_nlp = 2*n-1
m_nlp = 2+n
nlp_prob = ProblemMOI(n_nlp,m_nlp)

z_guess = zeros(2*n-1); #initial guess is all zeros
z_sol = ipopt_solve(z_guess,nlp_prob)

#We'll use the IPOPT solution as our reference for the following parts
y_ref = z_sol[1:n];
ẏ_ref = z_sol[(n+1):end];


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1323
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       49
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [13]:
#Plot Solution
plot(y_ref,label = "IPOPT Solution")
ylabel!("Y")
xlabel!("X")

## 2B: Solve with Newton's method (15 pts)

In [14]:
#Now we're going to implement the local solution technique based on the discrete Euler-Lagrange Equation

#One-step Discrete Lagrangian
function Gk(yk,yn,λ)
    
    ### Solution
    ym = 0.5*(yk+yn)
    ẏm = (yn-yk)/Δx
    return (ρ*g*ym+λ)*sqrt(1 + ẏm^2)*Δx
end

#Discrete Euler-Lagrange Equation
function DEL(ym,yk,yn,λ)
    
    ### Solution
    ForwardDiff.derivative(dy->Gk(ym,dy,λ),yk) + ForwardDiff.derivative(dy->Gk(dy,yn,λ),yk)
end

DEL (generic function with 1 method)

In [15]:
@testset "discrete EL" begin 
    yk = 4.3
    yn = 4.6
    λ = 3.0
    
    G1 = Gk(yk,yn,λ)
    
    # delete 
#     test_dict["G1"] = G1
    
    @test isapprox(G1, test_dict["G1"],atol = 1e-10)
    
    ym = 4.1
    del1 = DEL(ym,yk,yn,λ)
    
    # delete 
#     test_dict["del1"] = del1
    
    @test isapprox(del1, test_dict["del1"],atol = 1e-10)
end

Test Summary: | Pass  Total
discrete EL   |    2      2


Test.DefaultTestSet("discrete EL", Any[], 2, false, false)

In [16]:
#We need the Lagrange multiplier for the length constraint. We could get this out of IPOPT, but let's
#solve for it ourselves using the reference solution for y and the discrete Euler-Lagrange equation.

λ = 0.0

### Solution: There are several ways to do this knowing that the Lagrangian is always linear in the multiplier
r = DEL(y_ref[1],y_ref[2],y_ref[3],λ)
dr = ForwardDiff.derivative(dλ->DEL(y_ref[1],y_ref[2],y_ref[3],dλ), λ)
λ = -dr\r

52.36714674514151

In [17]:
@testset "lagrange multiplier" begin 
    @test isapprox(λ,52.367146716, atol = 1e-5 )
end

Test Summary:       | Pass  Total
lagrange multiplier |    1      1


Test.DefaultTestSet("lagrange multiplier", Any[], 1, false, false)

In [18]:
#Now that we know λ, y[1], and y[2] from the reference solution, let's use the DEL to recover the rest of the solution y[3:n]

ytraj = zeros(n)
ytraj[1:2] = copy(y_ref[1:2]) #use initial conditions from reference solution
for k = 2:(n-1)
    
    # TODO: use newtons method to solve for ytraj[k+1] given ytraj[k] and ytraj[k-1]
    # hint: r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1],λ) = 0 
    
    ### Solution
    ytraj[k+1] = copy(ytraj[k]) #initial guess
    r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1],λ) #residual
    while abs(r) > 1e-12
        dr = ForwardDiff.derivative(yn->DEL(ytraj[k-1],ytraj[k],yn,λ),ytraj[k+1])
        ytraj[k+1] = ytraj[k+1] - dr\r
        r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1],λ)
    end
end

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref,label = "IPOPT")
plot!(LinRange(0,w,n),ytraj,label = "Newton")
xlabel!("X")
ylabel!("Y")

In [19]:
@testset "DEL vs IPOPT" begin 
    @test maximum(abs.(ytraj-y_ref)) < 1e-5
end

Test Summary: | Pass  Total
DEL vs IPOPT  |    1      1


Test.DefaultTestSet("DEL vs IPOPT", Any[], 1, false, false)

## 2C: Solve as a continuous ODE (20 pts)

In [20]:
#Now let's implement the continuous ODE formulation of the problem.

#Continuous Lagrangian
function G(y,ẏ,λ)
    
    ### Solution
    (ρ*g*y+λ)*sqrt(1 + ẏ^2)
end

#Continuous Euler-Lagrange Equation
function EL(y,ẏ,λ)
    
    ### Solution
    DG = ForwardDiff.derivative(dy->G(dy,ẏ,λ),y)
    DDG = ForwardDiff.hessian(dx->G(dx[1],dx[2],λ),[y; ẏ])
    
    ÿ = DDG[2,2]\(DG - DDG[1,2]*ẏ)
end

EL (generic function with 1 method)

In [21]:
@testset "continous EL" begin 
    y = 3.4
    ẏ = -0.1 
    λ = 3.2 
    
    G2 = G(y,ẏ,λ)
    el2 = EL(y,ẏ,λ)
    
    # delete 
#     test_dict["G2"] = G2
#     test_dict["el2"] = el2
    
    @test isapprox(G2, test_dict["G2"],atol = 1e-10)
    @test isapprox(el2, test_dict["el2"],atol = 1e-10)
end

Test Summary: | Pass  Total
continous EL  |    2      2


Test.DefaultTestSet("continous EL", Any[], 2, false, false)

In [22]:
#Let's integrate the continuous version using DifferentialEquations.jl

#wrapper function for DifferentialEquations.jl interface 
function f(x,p,t)
    y = x[1]
    ẏ = x[2]
    ÿ = EL(y,ẏ,λ)
    return [ẏ; ÿ]
end

x0 = [y_ref[1]; ẏ_ref[1]]
tspan = (0.0,w)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),reltol=1e-8,abstol=1e-8);

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref,label = "IPOPT Solution")
plot!(sol, vars=1,label = "DifferentialEquations.jl Solution v1")
xlabel!("X")
ylabel!("Y")

┌ Warning: To maintain consistency with solution indexing, keyword argument vars will be removed in a future version. Please use keyword argument idxs instead.
│   caller = ip:0x0
└ @ Core :-1


In [23]:
#Let's also integrate the continuous solution using implicit midpoint
xtraj1 = zeros(2,n)
xtraj1[:,1] .= x0

# TODO: Implicit midpoint step (we did this in the last homework)
function implicit_mid(xk)
    
    ### Solution
    xn = copy(xk)
    xm = 0.5*(xk + xn)
    r = xn - xk - Δx*f(xm,0,0)
    while maximum(abs.(r)) > 1e-12
        dr = I - 0.5*Δx*ForwardDiff.jacobian(dx->f(dx,0,0),xm)
        xn = xn - dr\r
        xm = 0.5*(xk + xn)
        r = xn - xk - Δx*f(xm,0,0)
    end
    
    return xn
end

#Simulate with implicit midpoint
for k = 1:(n-1)
    xtraj1[:,k+1] .= implicit_mid(xtraj1[:,k])
end

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref,label = "IPOPT Solution")
plot!(LinRange(0,w,n),xtraj1[1,:],label = "Midpoint Solution v1")
xlabel!("X")
ylabel!("Y")

In [24]:
#Remember that ẏ[1] from our reference solution is a midpoint approximation, so it's actually ẏ(Δx/2), not ẏ(0).
#Let's try fixing this by taking a backwards step by Δx/2. You can use explicit Euler for this.

### Solution (using a half step of Euler - other methods are fine too)
ẏm = ẏ_ref[1]
ym = 0.5*(y_ref[1]+y_ref[2]) 
xm = [ym; ẏm]
x0 = xm - 0.5*Δx*f(xm,0,0)

2-element Vector{Float64}:
 -5.551115123125783e-17
 -2.667469486306863

In [25]:
@testset "half a step backwards" begin 
    @test isapprox(x0,[0;-2.667469488054186],atol = 1e-4)
end

Test Summary:         | Pass  Total
half a step backwards |    1      1


Test.DefaultTestSet("half a step backwards", Any[], 1, false, false)

In [26]:
#Let's try integrating with DifferentialEquations.jl again with this new x0
tspan = (0.0,w)
prob = ODEProblem(f,x0,tspan)
sol2 = solve(prob,Tsit5(),reltol=1e-8,abstol=1e-8);

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref,label = "IPOPT Solution")
plot!(sol2, vars=1,label = "DifferentialEquations.jl Solution v2")
xlabel!("X")
ylabel!("Y")

In [27]:
#Now let's try integrating with implicit midpoint again
xtraj2 = zeros(2,n)
xtraj2[:,1] = x0

#Simulate with implicit midpoint
for k = 1:(n-1)
    xtraj2[:,k+1] .= implicit_mid(xtraj2[:,k])
end

#Compare against reference IPOPT solution
plot(y_ref,label = "IPOPT Solution")
plot!(xtraj2[1,:],label = "Midpoint Solution v2")
xlabel!("X")
ylabel!("Y")

In [28]:
#Check error vs. IPOPT reference solution
@testset "final error" begin 
    @test maximum(abs.(xtraj2[1,:]-y_ref)) < 1e-4
end

Test Summary: | Pass  Total
final error   |    1      1


Test.DefaultTestSet("final error", Any[], 1, false, false)

In [29]:
# using FileIO
# save(joinpath(jld2path,"q2.jld2"),test_dict)